In [ ]:
pip install aiohttp

In [22]:
import time
import nest_asyncio
import requests
import json
from urllib import request
import multiprocessing
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor

In [23]:
#Synchronous
def download_images():
    response = requests.get("https://picsum.photos/v2/list")
    if response.status_code != 200:
        raise AttributeError('GET /tasks/ {}'.format(response.status_code))
    data = json.loads(response.text)[:10]

    pictures=[]
    for s in data:
        pictures.append(s['download_url']+".jpg")
    return pictures

def saveImages(link):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    request.urlretrieve(link, "picture/{}.{}".format(filename, fileformat))

def main():
    images = download_images()
    for image in images:
        saveImages(image)

start_time = time.time()
main()
duration_synch = time.time() - start_time
print(f"Time taken: {duration_synch}")

Time taken: 8.019447088241577


In [12]:
#Multithreading
def process_images_threading():
    images = download_images()
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(saveImages,images)

start_time = time.time()
process_images_threading()
duration_multithreading = time.time() - start_time
print(f"Time taken: {duration_multithreading}")

Time taken: 5.275107383728027


In [13]:
#Asyncio
async def download_images_asyncio(link, session):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    async with session.get(link) as response:
        with open("picture/{}.{}".format(filename, fileformat), 'wb') as fd:
            async for data in response.content.iter_chunked(1024):
                fd.write(data)

async def main_asyncio():
    images = download_images()

    async with aiohttp.ClientSession() as session:
        tasks=[download_images_asyncio(image,session)for image in images]
        return await asyncio.gather(*tasks)

start_time = time.time()
nest_asyncio.apply()
asyncio.run(main_asyncio())
duration_asyncio = time.time() - start_time
print(f"Time taken: {duration_asyncio}")

Time taken: 1.086766004562378


In [35]:
#Compare
print(f"Synchronous: {duration_synch}")
print(f"Multithreading: {duration_multithreading}")
print(f"Asyncio: {duration_asyncio}")

Synchronous: 8.019447088241577
Multithreading: 5.275107383728027
Asyncio: 1.086766004562378
